# Demo of xesmf failure to regrid lazily

In [1]:
# create two global grids
import xesmf as xe
grid_a = xe.util.grid_global(1, 1)
grid_b = xe.util.grid_global(2, 2)

In [10]:
grid_a

<xarray.Dataset>
Dimensions:  (y: 180, x: 360, y_b: 181, x_b: 361)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) int64 -180 -179 -178 -177 -176 ... 176 177 178 179 180
    lat_b    (y_b, x_b) int64 -90 -90 -90 -90 -90 -90 -90 ... 90 90 90 90 90 90
Dimensions without coordinates: y, x, y_b, x_b
Data variables:
    *empty*

In [7]:
len(grid_a.lat)

180

In [8]:
# create a regridder
regridder = xe.Regridder(grid_a, grid_b, 'bilinear')

In [9]:
regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_180x360_90x180.nc 
Reuse pre-computed weights? False 
Input grid shape:           (180, 360) 
Output grid shape:          (90, 180) 
Periodic in longitude?      False

In [13]:
# regrid a numpy dummy array
import xarray as xr
import numpy as np

nx, ny = len(grid_a.x), len(grid_a.y)

data = xr.DataArray(np.random.rand(ny,nx), dims=['lat','lon'])
data_regridded = regridder(data)
data_regridded

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:598: UserWarning: Using dimensions ('lat', 'lon') from data variable None as the horizontal dimensions for the regridding.
  warnings.warn(


<xarray.DataArray (y: 90, x: 180)>
array([[0.41472051, 0.58704265, 0.69692102, ..., 0.64557168, 0.56726411,
        0.49148917],
       [0.607985  , 0.38921592, 0.35737876, ..., 0.55413717, 0.52364932,
        0.4388525 ],
       [0.68694008, 0.44111959, 0.46890487, ..., 0.41499019, 0.57716766,
        0.34638168],
       ...,
       [0.41074175, 0.31333601, 0.60212183, ..., 0.16893205, 0.48318922,
        0.656606  ],
       [0.48829551, 0.37166897, 0.27672405, ..., 0.71945413, 0.48623656,
        0.3966852 ],
       [0.48093302, 0.46295733, 0.51381591, ..., 0.70688502, 0.19424176,
        0.445732  ]])
Coordinates:
    lon      (y, x) float64 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
    lat      (y, x) float64 -89.0 -89.0 -89.0 -89.0 ... 89.0 89.0 89.0 89.0
Dimensions without coordinates: y, x
Attributes:
    regrid_method:  bilinear

This works as expected, but now lets do this to a dask array

In [19]:
data_dask = data.chunk({'lon':-1})
data_dask_regridded = regridder(data_dask)
data_dask_regridded.load()

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:598: UserWarning: Using dimensions ('lat', 'lon') from data variable None as the horizontal dimensions for the regridding.
  warnings.warn(


<xarray.DataArray (y: 90, x: 180)>
array([[0.41472051, 0.58704265, 0.69692102, ..., 0.64557168, 0.56726411,
        0.49148917],
       [0.607985  , 0.38921592, 0.35737876, ..., 0.55413717, 0.52364932,
        0.4388525 ],
       [0.68694008, 0.44111959, 0.46890487, ..., 0.41499019, 0.57716766,
        0.34638168],
       ...,
       [0.41074175, 0.31333601, 0.60212183, ..., 0.16893205, 0.48318922,
        0.656606  ],
       [0.48829551, 0.37166897, 0.27672405, ..., 0.71945413, 0.48623656,
        0.3966852 ],
       [0.48093302, 0.46295733, 0.51381591, ..., 0.70688502, 0.19424176,
        0.445732  ]])
Coordinates:
    lon      (y, x) float64 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
    lat      (y, x) float64 -89.0 -89.0 -89.0 -89.0 ... 89.0 89.0 89.0 89.0
Dimensions without coordinates: y, x
Attributes:
    regrid_method:  bilinear